In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
np.set_printoptions(precision=3)

with open('winequality-white.csv', 'r') as f:
  temp = np.genfromtxt(f,delimiter=',', skip_header=1)
  X = temp[:,:-1]
  y = temp[:,-1]

In [52]:
from sklearn.preprocessing import OneHotEncoder

y = y.reshape((-1,1))
print(y.shape)
enc = OneHotEncoder()
enc.fit(y)
#print(enc.transform(y).toarray())
y_enc = enc.transform(y).toarray()
print(y_enc.shape)

(4898, 1)
(4898, 7)


In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X)
print(X_scaled)

[[ 1.721e-01 -8.327e-02  2.133e-01 ... -1.247e+00 -3.492e-01 -1.393e+00]
 [-6.575e-01  2.141e-01  4.800e-02 ...  7.400e-01  1.342e-03 -8.243e-01]
 [ 1.476e+00  1.584e-02  5.438e-01 ...  4.751e-01 -4.368e-01 -3.367e-01]
 ...
 [-4.205e-01 -3.806e-01 -1.192e+00 ... -1.313e+00 -2.616e-01 -9.055e-01]
 [-1.606e+00  1.150e-01 -2.826e-01 ...  1.005e+00 -9.626e-01  1.858e+00]
 [-1.013e+00 -6.779e-01  3.786e-01 ...  4.751e-01 -1.488e+00  1.045e+00]]


In [54]:
X_presplit, X_test, y_presplit, y_test = train_test_split(X_scaled,y_enc,test_size=0.2,train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_presplit,y_presplit,test_size=0.25,train_size=0.75)

In [55]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(11, input_dim=11, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(7, activation='softmax')])
X.shape

(4898, 11)

In [56]:
net.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [57]:
#net.fit(X_train, y_train, epochs=150, batch_size=10)
net.fit(X_train, y_train, epochs=100, batch_size = 16, validation_data=(X_val, y_val))

Epoch 1/100
184/184 [==============================] - 0s 1ms/step - loss: 0.5450 - accuracy: 0.2379 - val_loss: 0.3669 - val_accuracy: 0.4214
Epoch 2/100
184/184 [==============================] - 0s 710us/step - loss: 0.3238 - accuracy: 0.4506 - val_loss: 0.3130 - val_accuracy: 0.4510
Epoch 3/100
184/184 [==============================] - 0s 680us/step - loss: 0.3053 - accuracy: 0.4530 - val_loss: 0.3033 - val_accuracy: 0.4510
Epoch 4/100
184/184 [==============================] - 0s 686us/step - loss: 0.2972 - accuracy: 0.4564 - val_loss: 0.2966 - val_accuracy: 0.4490
Epoch 5/100
184/184 [==============================] - 0s 684us/step - loss: 0.2909 - accuracy: 0.4629 - val_loss: 0.2923 - val_accuracy: 0.4551
Epoch 6/100
184/184 [==============================] - 0s 705us/step - loss: 0.2864 - accuracy: 0.4717 - val_loss: 0.2874 - val_accuracy: 0.4571
Epoch 7/100
184/184 [==============================] - 0s 702us/step - loss: 0.2822 - accuracy: 0.4779 - val_loss: 0.2833 - val_accu

In [77]:
# Test the model after training
test_results = net.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
y_pred = np.argmax(net.predict(X_test), axis=-1)
print(y_pred.shape)
print(y_val.shape)
print(y_val)

b = (np.argmax(y_val, axis = 1) + 3)

print(tf.math.confusion_matrix(y_pred, b)[:-3,3:])

31/31 [==============================] - 0s 551us/step - loss: 0.2509 - accuracy: 0.5684
Test results - Loss: 0.25092700123786926 - Accuracy: 0.5683673620223999%
31/31 [==============================] - 0s 367us/step
(980,)
(980, 7)
[[3. 4. 3. ... 3. 3. 3.]
 [3. 3. 3. ... 4. 3. 3.]
 [3. 3. 3. ... 3. 3. 3.]
 ...
 [3. 3. 4. ... 3. 3. 3.]
 [3. 3. 3. ... 3. 3. 3.]
 [3. 3. 3. ... 3. 3. 3.]]
tf.Tensor(
[[  0   0   0   0   0   0   0]
 [  0   0   1   7   4   0   0]
 [  1  10  75 133  46  12   1]
 [  1  16 172 244 108  27   0]
 [  1   5  40  58  14   2   2]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]], shape=(7, 7), dtype=int32)


In [80]:
net2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, input_dim=11, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(7, activation='softmax')])
net2.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
net2.fit(X_train, y_train, epochs=100, batch_size = 16, validation_data=(X_val, y_val))

Epoch 1/100
184/184 [==============================] - 1s 1ms/step - loss: 0.5224 - accuracy: 0.2236 - val_loss: 8.0688 - val_accuracy: 0.4510
Epoch 2/100
184/184 [==============================] - 0s 794us/step - loss: 0.3145 - accuracy: 0.4530 - val_loss: 8.9583 - val_accuracy: 0.4510
Epoch 3/100
184/184 [==============================] - 0s 747us/step - loss: 0.3045 - accuracy: 0.4530 - val_loss: 8.6703 - val_accuracy: 0.4510
Epoch 4/100
184/184 [==============================] - 0s 696us/step - loss: 0.3017 - accuracy: 0.4530 - val_loss: 9.3043 - val_accuracy: 0.4510
Epoch 5/100
184/184 [==============================] - 0s 682us/step - loss: 0.2998 - accuracy: 0.4530 - val_loss: 9.5670 - val_accuracy: 0.4510
Epoch 6/100
184/184 [==============================] - 0s 719us/step - loss: 0.2957 - accuracy: 0.4554 - val_loss: 10.4080 - val_accuracy: 0.4622
Epoch 7/100
184/184 [==============================] - 0s 767us/step - loss: 0.2893 - accuracy: 0.4711 - val_loss: 10.5177 - val_ac

In [79]:
# Test the model after training
test_results = net2.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
y_pred = np.argmax(net.predict(X_test), axis=-1)
print(y_pred.shape)
print(y_val.shape)
print(y_val)

b = (np.argmax(y_val, axis = 1) + 3)

print(tf.math.confusion_matrix(y_pred, b)[:-3,3:])

31/31 [==============================] - 0s 641us/step - loss: 0.2613 - accuracy: 0.5439
Test results - Loss: 0.26134181022644043 - Accuracy: 0.5438775420188904%
31/31 [==============================] - 0s 366us/step
(980,)
(980, 7)
[[3. 4. 3. ... 3. 3. 3.]
 [3. 3. 3. ... 4. 3. 3.]
 [3. 3. 3. ... 3. 3. 3.]
 ...
 [3. 3. 4. ... 3. 3. 3.]
 [3. 3. 3. ... 3. 3. 3.]
 [3. 3. 3. ... 3. 3. 3.]]
tf.Tensor(
[[  0   0   0   0   0   0   0]
 [  0   0   1   7   4   0   0]
 [  1  10  75 133  46  12   1]
 [  1  16 172 244 108  27   0]
 [  1   5  40  58  14   2   2]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]], shape=(7, 7), dtype=int32)


In [ ]:
#Vanishing gradients: Yes in net 2, too many layers possibly